In [1]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [2]:

loader = PyPDFLoader('./data/uae_driving.pdf')
data = loader.load()

In [3]:
print(type(data))
print(len(data))
print(type(data[0]))

<class 'list'>
178
<class 'langchain_core.documents.base.Document'>


In [4]:
data[0]

Document(page_content='Light\nMotor Vehicle \nHandbook\nA Guide to Safe Driving\nLicensing Agency\n10th Edition\n8009090    rta.ae\n', metadata={'source': './data/uae_driving.pdf', 'page': 0})

In [5]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

In [6]:
print(type(docs))
print(len(docs))
print(type(docs[0]))

<class 'list'>
369
<class 'langchain_core.documents.base.Document'>


In [7]:
#embeddin model loading- local mode 

# Define the path to the pre-trained model you want to use
modelPath = "C://Users//shana//OneDrive//Desktop//Generative_AI//all-MiniLM-L6-v2"


# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [8]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
print(len(query_result))
print(query_result[:3])

384
[-0.03833857923746109, 0.1234646886587143, -0.02864295430481434]


In [9]:
db = FAISS.from_documents(docs, embeddings)

question = "adaptive cruise control"
searchDocs = db.similarity_search(question)
#print(searchDocs[0].page_content)

In [10]:
print(type(searchDocs))
print(len(searchDocs))
print(type(searchDocs[0]))

<class 'list'>
4
<class 'langchain_core.documents.base.Document'>


In [11]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [13]:
#loading tokernizer and model-local mode 
model_name = "C://Users//shana//OneDrive//Desktop//Generative_AI//Phi-3-mini-4k-instruct"


torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")




In [13]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [18]:
question = "what are control signs?"
result = qa.run({"query": question})
print(result["result"])

ValueError: Context information is below. 
------------
travel. Here are some examples of control signs:
You must give way to 
pedestrians
Give way to 
cyclistsYou must 
give way 
You must go 
this wayYou must not enter 
 this road
You must go 
this wayYou must 
stop
You must go 
this way
111
Light Motor Vehicle Handbook  -  A Guide to Safe Driving
------------
Given the context information and not prior knowledge, answer the question: what are control signs?
 argument needs to be of type (SquadExample, dict)